# 9 LangChain 없이 RAG 파이프라인 구현

In [1]:
!uv add -q python-docx tiktoken 

In [3]:
from docx import Document

# 워드 문서를 직접 로드한다.
document = Document('../documents/law_markdown.docx')

full_text = ''
for index, paragraph in enumerate(document.paragraphs):
   full_text += f'{paragraph.text}\n'


In [4]:
import tiktoken

# 텍스트를 청크로 분할하는 함수를 직접 구현해야 한다.
def split_text(full_text, chunk_size):
   # 토크나이저를 직접 초기화해야 한다.
   # LangChain은 이런 저수준 설정을 자동으로 처리해준다.
   encoder = tiktoken.encoding_for_model("gpt-4o")
   
   # 전체 텍스트를 토큰으로 인코딩한다.
   # LangChain은 다양한 인코딩 방식을 지원하지만, 여기서는 수동으로 처리해야 한다.
   total_encoding = encoder.encode(full_text)
   total_token_count = len(total_encoding)
   
   # 청크 단위로 텍스트를 분할한다.
   text_list = []
   for i in range(0, total_token_count, chunk_size):
       chunk = total_encoding[i: i+chunk_size]
       decoded = encoder.decode(chunk)
       text_list.append(decoded)
  
   return text_list

# 청크 크기를 1500으로 설정하여 분할한다.
chunk_list = split_text(full_text, 1500)


In [5]:
!uv add chromadb

Resolved 187 packages in 40ms                                        
Audited 165 packages in 0.04ms                                       


In [ ]:
import chromadb

# python-chroma 패키지를 활용한 클라이언트 선언
chroma_client = chromadb.Client()

In [7]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

# 임베딩 함수를 직접 설정해야 한다.
openai_embedding = OpenAIEmbeddingFunction(
                                 api_key=openai_api_key,
                                 model_name='text-embedding-3-large')

# 컬렉션을 직접 생성하고 관리해야 한다.
# LangChain은 벡터 스토어 생성과 관리를 추상화하여 제공하지만, 여기서는 저수준 API를 직접 다뤄야 한다.
collection_name = 'tax_collection'
tax_collection = chroma_client.get_or_create_collection(
                         collection_name,
                         embedding_function=openai_embedding)


In [8]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')


In [ ]:
# 벡터 저장소에 데이터 추가
tax_collection.add(documents=chunk_list, ids=id_list)

In [ ]:
question = "10억짜리 집을 2채 가지고 있을 때 세금을 얼마나 내나요?"
retrieved_doc = tax_collection.query(
                    query_texts=question,  # 벡터 저장소 검색을 위한 질문    
                    n_results=1            # 벡터 저장소에서 불러올 문서 개수
                )
retrieved_doc

{'ids': [['1']],
 'embeddings': None,
 'documents': [['. 6. 9.>\n\n제2장 주택에 대한 과세\n\n제7조(납세의무자) ①과세기준일 현재 주택분 재산세의 납세의무자는 종합부동산세를 납부할 의무가 있다. <개정 2005. 12. 31., 2008. 12. 26., 2020. 8. 18.>\n② 「신탁법」 제2조에 따른 수탁자(이하 “수탁자”라 한다)의 명의로 등기 또는 등록된 신탁재산으로서 주택(이하 “신탁주택”이라 한다)의 경우에는 제1항에도 불구하고 같은 조에 따른 위탁자(「주택법」 제2조제11호가목에 따른 지역주택조합 및 같은 호 나목에 따른 직장주택조합이 조합원이 납부한 금전으로 매수하여 소유하고 있는 신탁주택의 경우에는 해당 지역주택조합 및 직장주택조합을 말한다. 이하 “위탁자”라 한다)가 종합부동산세를 납부할 의무가 있다. 이 경우 위탁자가 신탁주택을 소유한 것으로 본다.<신설 2020. 12. 29.>\n③ 삭제<2008. 12. 26.>\n[2008. 12. 26. 법률 제9273호에 의하여 2008. 11. 13. 헌법재판소에서 위헌 결정된 이 조를 개정함.]\n\n제7조의2(신탁주택 관련 수탁자의 물적납세의무) 신탁주택의 위탁자가 다음 각 호의 어느 하나에 해당하는 종합부동산세 또는 강제징수비(이하 “종합부동산세등”이라 한다)를 체납한 경우로서 그 위탁자의 다른 재산에 대하여 강제징수를 하여도 징수할 금액에 미치지 못할 때에는 해당 신탁주택의 수탁자는 그 신탁주택으로써 위탁자의 종합부동산세등을 납부할 의무가 있다.\n1. 신탁 설정일 이후에 「국세기본법」 제35조제2항에 따른 법정기일이 도래하는 종합부동산세로서 해당 신탁주택과 관련하여 발생한 것\n2. 제1호의 금액에 대한 강제징수 과정에서 발생한 강제징수비\n[본조신설 2020. 12. 29.]\n\n제8조(과세표준) ① 주택에 대한 종합부동산세의 과세표준은 납세의무자별로 주택의 공시가격을 합산한 금액에서 다음 각 호의 금액을 공제한 금액에 

In [13]:
retrieved_doc['documents'][0]

['. 6. 9.>\n\n제2장 주택에 대한 과세\n\n제7조(납세의무자) ①과세기준일 현재 주택분 재산세의 납세의무자는 종합부동산세를 납부할 의무가 있다. <개정 2005. 12. 31., 2008. 12. 26., 2020. 8. 18.>\n② 「신탁법」 제2조에 따른 수탁자(이하 “수탁자”라 한다)의 명의로 등기 또는 등록된 신탁재산으로서 주택(이하 “신탁주택”이라 한다)의 경우에는 제1항에도 불구하고 같은 조에 따른 위탁자(「주택법」 제2조제11호가목에 따른 지역주택조합 및 같은 호 나목에 따른 직장주택조합이 조합원이 납부한 금전으로 매수하여 소유하고 있는 신탁주택의 경우에는 해당 지역주택조합 및 직장주택조합을 말한다. 이하 “위탁자”라 한다)가 종합부동산세를 납부할 의무가 있다. 이 경우 위탁자가 신탁주택을 소유한 것으로 본다.<신설 2020. 12. 29.>\n③ 삭제<2008. 12. 26.>\n[2008. 12. 26. 법률 제9273호에 의하여 2008. 11. 13. 헌법재판소에서 위헌 결정된 이 조를 개정함.]\n\n제7조의2(신탁주택 관련 수탁자의 물적납세의무) 신탁주택의 위탁자가 다음 각 호의 어느 하나에 해당하는 종합부동산세 또는 강제징수비(이하 “종합부동산세등”이라 한다)를 체납한 경우로서 그 위탁자의 다른 재산에 대하여 강제징수를 하여도 징수할 금액에 미치지 못할 때에는 해당 신탁주택의 수탁자는 그 신탁주택으로써 위탁자의 종합부동산세등을 납부할 의무가 있다.\n1. 신탁 설정일 이후에 「국세기본법」 제35조제2항에 따른 법정기일이 도래하는 종합부동산세로서 해당 신탁주택과 관련하여 발생한 것\n2. 제1호의 금액에 대한 강제징수 과정에서 발생한 강제징수비\n[본조신설 2020. 12. 29.]\n\n제8조(과세표준) ① 주택에 대한 종합부동산세의 과세표준은 납세의무자별로 주택의 공시가격을 합산한 금액에서 다음 각 호의 금액을 공제한 금액에 부동산 시장의 동향과 재정 여건 등을 고려하여 100분의 60부터 100분의 100까지의 범위에

In [ ]:
from openai import OpenAI

# OpenAI 클라이언트를 직접 초기화해야 한다. 여기서는 OpenAI 전용 코드를 작성해야 한다.
client = OpenAI()

# API 호출을 직접 구성하고 실행해야 한다.
# LangChain은 프롬프트 관리, 콘텍스트 주입, 출력 파싱 등을 체계적으로 처리해주지만,
# 여기서는 모든 것을 수동으로 구성해야 한다.
response = client.chat.completions.create(
 model="gpt-4o",
 messages=[
   {"role": "system", "content": f"종합부동산세 전문가입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요 {retrieved_doc['documents'][0]}"},
   {"role": "user", "content": question}
 ]
)

# 응답에서 필요한 내용을 직접 추출해야 한다.
# LangChain은 출력 파서를 통해 응답 처리를 자동화해주지만,
# 여기서는 응답 객체의 구조를 직접 다뤄야 한다.
response.choices[0].message.content


'귀하가 10억 원짜리 주택을 2채 소유하고 있다면, 종합부동산세를 계산할 때 과세표준은 주택의 공시가격 총합에서 일정 금액을 공제한 후 공정시장가액비율을 적용하여 산출됩니다.\n\n1. **주택의 공시가격 합산**: 두 주택의 합산된 공시가격이 20억 원입니다.\n\n2. **과세표준 공제**: 귀하가 1세대 1주택자가 아니라면, 9억 원이 공제됩니다.\n\n3. **공정시장가액비율 적용**: 현재의 공정시장가액비율은 법에 따라 60%에서 100%의 범위 내의 비율이 적용됩니다. 예를 들어, 60%의 비율을 가정할 경우, 과세표준은 (20억 원 - 9억 원) × 60% = 6.6억 원이 됩니다.\n\n4. **세율 적용**: 과세표준에 따라 적용되는 세율은 종합부동산세법에 명시된 세율에 따라 달라집니다. 여러 구간의 세율이 있으므로 정확한 세율은 구체적인 과세표준 금액과 법령에 따라 계산되며, 여기서 제시하기는 어렵습니다.\n\n정확한 세율과 세금 계산을 위해서는 전문가의 도움이 필요할 수 있습니다. 공정시장가액비율과 구체적인 세율은 귀하가 과세되는 시점의 법률에 따라 달라질 수 있으므로 세무 전문가와 상담하시기를 권장합니다.'